In [44]:
%pip install BeautifulSoup
%pip install langchain
%pip install openai
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\manumishra\AppData\Local\Temp\pip-install-edxsmta1\beautifulsoup_92300290bf774d90b1b6699ef0f5f00f\setup.py", line 3
          "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                         ^^
      SyntaxError: invalid syntax
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------- 635.3/635.3 kB 13.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [17]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



BrightSox


In [36]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def get_html(url):
    response = requests.get(url)

    # Ensure the request was successful
    if response.status_code != 200:
        print(f"Failed to get page")
        return None

    return response.text

def extract_p_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    p_text = [tag.get_text().strip() for tag in soup.body.find_all('p') if tag.get_text().strip()]
    return p_text

def extract_a_tags(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    a_tags = []
    
    for tag in soup.body.find_all('a'):
        text = tag.get_text().strip()
        href = tag.get('href')
        
        # Only consider tags with non-blank text and href
        if not text or not href:
            continue
        
        # Parse the URL
        parsed_url = urlparse(href)
        
        # Exclude page anchors
        if parsed_url.path.startswith('#'):
            continue

        # Include relative links and absolute links that belong to the original domain
        if parsed_url.netloc == '' or parsed_url.netloc == urlparse(base_url).netloc:
            a_tags.append({'href': href, 'text': text})
    
    return a_tags

def extract_information(url):
    html = get_html(url)
    if not html:
        return None
    
    p_tags = extract_p_tags(html)
    a_tags = extract_a_tags(html, url)
    
    additional_p_tags = []
    
    for a_tag in a_tags[:min(10, len(a_tags))]:
        additional_html = get_html(a_tag['href'])
        if additional_html:
            additional_p_tags.extend(extract_p_tags(additional_html))
    
    combined_text = ' '.join(p_tags + additional_p_tags)
    return combined_text



In [37]:
# Call the functions with the URL
url = 'https://www.solsunless.com/custom-airbrush-tanning'
information = extract_information(url)
print(information)


HOME ABOUT SERVICES LOCATIONS SPECIALS JOIN OUR TEAM PREP MY MEMBERSHIP More Our process is very unique as we offer a “customized color” that is best suited to your skin color/type. Our process does not involve spraying everyone the same color; our certified spray tech will always mix a customized color after she views your skin and has discussed your desired results. You will walk out of our Salon absolutely glowing and feeling transformed. Double Bronze not included with sessions. ONE ​ FIVE ​ TEN ​ DOUBLE BRONZE (PER SESSION) ​ RAPID (PER SESSION) $55 ​ $225 ​ $350 ​ $10 ​ $15 TWO PER MONTH (double bronze included) ​ TWO PER MONTH W/RAPID (double bronze included) ​ UNLIMITED (double bronze included) ​ UNLIMITED W/RAPID (double bronze included) $69 ​ ​ $79 ​ ​ $109 ​ ​ $129 ​ $20 enrollment fee required. See store for details. SIX MONTHS UNLIMITED (double bronze included) ​ TWELVE MONTHS UNLIMITED (double bronze included) $480 ​ ​ $820 ​ No enrollment fee required. See store for deta

## Setup Chain

In [45]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)
template = """
I want to extract information like name of business, what does it do, list of services offered and price of different options of those services from the following text in json format.
{TextFromWebSite}?
"""

prompt = PromptTemplate(
    input_variables=["TextFromWebSite"],
    template=template,
)

chain = LLMChain(llm=llm, prompt=prompt)
chain.run(information)

'\n  {\n    "Business Name": "Sol Sunless Tanning, LLC",\n    "Business Description": "We offer a “customized color” that is best suited to your skin color/type. Our process does not involve spraying everyone the same color; our certified spray tech will always\xa0mix a customized color after she views your skin and has discussed your desired results. You will walk out of our Salon absolutely glowing and feeling transformed.",\n    "Services Offered": [\n      {\n        "Name": "ONE",\n        "Price": "$55"\n      },\n      {\n        "Name": "FIVE",\n        "Price": "$225"\n      },\n      {\n        "Name": "TEN",\n        "Price": "$350"\n      },\n      {\n        "Name": "Double Bronze (Per Session)",\n        "Price": "$10"\n      },\n      {\n        "Name": "Rapid (Per Session)",\n        "Price": "$15"\n      },\n      {\n        "Name": "Two Per Month (double bronze included)",\n        "Price": "$69"\n      },\n      {\n        "Name'

In [48]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

from langchain.chains import AnalyzeDocumentChain

prompt_template = """From the following text, answer following questions in provided format:
Q1. What is the Name of the business?
Q2. What are the different Services offered by this business?
Q3. What are the different service packages with their price information and duration if any?

Text:
{text}

Result Format:
NameOfBusiness: [answer to Q1]
Services offered: [answer to Q2]
Packages:[answer to Q3]
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain, prompt=PROMPT)

summarize_document_chain.run(information)


" Sol Sunless Tanning provides custom tanning services tailored to each individual's skin color/type, with multiple locations in Texas. They offer various packages and specials, and are committed to providing the best customer experience."

In [52]:
url = 'https://www.solsunless.com/'
information = extract_information(url)

In [53]:
from langchain.chains.question_answering import load_qa_chain

prompt_template = """From the following text, answer following questions in provided Json format:
Q1. What is the Name of the business?
Q2. What are the different Services offered by this business?
Q3. What are the different service packages with their price information and duration if any?


Result Format:
NameOfBusiness: [answer to Q1]
Services offered: [answer to Q2] [name:]
Packages:[answer to Q3] format array of [name:, duration:, price:]
"""

qa_chain = load_qa_chain(llm, chain_type="map_reduce", verbose=True)
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
qa_document_chain.run(input_document=information, question=prompt_template)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
HOME ABOUT SERVICES LOCATIONS SPECIALS JOIN OUR TEAM PREP MY MEMBERSHIP More ©2019 by Sol Sunless Tanning, LLC HOME ABOUT SERVICES LOCATIONS SPECIALS JOIN OUR TEAM PREP MY MEMBERSHIP More ©2019 by Sol Sunless Tanning, LLC HOME ABOUT SERVICES LOCATIONS SPECIALS JOIN OUR TEAM PREP MY MEMBERSHIP More 214-705-3089 ​ 8837 Lebanon Road STE 600 Frisco, TX 75034 214-513-8200 ​ 2704 Cross Timbers Road STE 116 Flower Mound, TX 75028 ​ 214-484-4988 ​ 5555 E. Mockingbird Lane STE 100 Dallas, TX 75206 ​ 972-597-2868 ​ 2910 Cityplace West Blvd STE 120 Dallas, TX 75204 ​ 214-856-3633 ​ 5101McKinney Ranch PKWY STE 110 McKinney, TX 75070 ​ ​ Sunday..................... Monday.................... Tuesday................... Wednesday.............. Thursday.................. Friday......

' NameOfBusiness: Sol Sunless Tanning, LLC\nServices offered: No relevant text.\nPackages: No relevant text.'